# Importing and cleaning the dataset

### Importing libraries and the dataset to be cleaned

In [154]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from scipy.cluster import hierarchy
from scipy.interpolate import BSpline

from fancyimpute import KNN

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('seaborn-white')

In [155]:
# import the csv dataset as a pandas DataFrame
df = pd.read_csv('data/selected_countries.csv')
# change index (row labels)
df = df.set_index('Country Code', verify_integrity=True)
# display df with all its columns
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
df.head()

,Country Name,Population_2014,Population_2015,Population_2016,Expense_%GDP_2014,Expense_%GDP_2015,Expense_%GDP_2016,ForeignDirectInvestment_NetInflows_BoP_currentUS$_2014,ForeignDirectInvestment_NetInflows_BoP_currentUS$_2015,ForeignDirectInvestment_NetInflows_BoP_currentUS$_2016,GDPgrowth_annual%_2014,GDPgrowth_annual%_2015,GDPgrowth_annual%_2016,Inflation_ConsumerPrices_annual%_2014,Inflation_ConsumerPrices_annual%_2015,Inflation_ConsumerPrices_annual%_2016,ExportsGoodsServices_%GDP_2014,ExportsGoodsServices_%GDP_2015,ExportsGoodsServices_%GDP_2016,GDP_currentUS$_2014,GDP_currentUS$_2015,GDP_currentUS$_2016,GDP_constantUS$_2014,GDP_constantUS$_2015,GDP_constantUS$_2016,ImportsGoodsServices_%GDP_2014,ImportsGoodsServices_%GDP_2015,ImportsGoodsServices_%GDP_2016,AccessElectricity_%population_2014,AccessElectricity_%population_2015,AccessElectricity_%population_2016,FossilFuelEnergyConsumption_%total_2014,FossilFuelEnergyConsumption_%total_2015,FossilFuelEnergyConsumption_%total_2016,EnergyUse_KgEquivalentPerCapita_2014,EnergyUse_KgEquivalentPerCapita_2015,EnergyUse_KgEquivalentPerCapita_2016,CO2Emissions_kt_2014,CO2Emissions_kt_2015,CO2Emissions_kt_2016,PovertyHeadcountRatio$1.90aday_2011PPP_%population_2014,PovertyHeadcountRatio$1.90aday_2011PPP_%population_2015,PovertyHeadcountRatio$1.90aday_2011PPP_%population_2016,Export_Petroleum_US $_2014,Export_Petroleum_US $_2015,Export_Petroleum_US $_2016,Export_Tot_US $_2014,Export_Tot_US $_2015,Export_Tot_US $_2016,Import_Petroleum_US $_2014,Import_Petroleum_US $_2015,Import_Petroleum_US $_2016,Import_Tot_US $_2014,Import_Tot_US $_2015,Import_Tot_US $_2016,AnnualHouseholdExpenditure_Gas_2010,AnnualHouseholdExpenditure_Electricity_2010,AnnualHouseholdExpenditure_Other fuels_2010,FinalConsumptionExpenditure_currentUS$_2014,FinalConsumptionExpenditure_currentUS$_2015,FinalConsumptionExpenditure_currentUS$_2016,FinalConsumptionExpenditure_constantUS$_2014,FinalConsumptionExpenditure_constantUS$_2015,FinalConsumptionExpenditure_constantUS$_2016,GNI_currentUS$_2014,GNI_currentUS$_2015,GNI_currentUS$_2016,GNI_constantUS$_2014,GNI_constantUS$_2015,GNI_constantUS$_2016,ImportsOfGoodsAndServices_currentUS$_2014,ImportsOfGoodsAndServices_currentUS$_2015,ImportsOfGoodsAndServices_currentUS$_2016,ImportsOfGoodsAndServices_constantUS$_2014,ImportsOfGoodsAndServices_constantUS$_2015,ImportsOfGoodsAndServices_constantUS$_2016,HouseholdsFinalConsumptionExpenditure_currentUS$_2014,HouseholdsFinalConsumptionExpenditure_currentUS$_2015,HouseholdsFinalConsumptionExpenditure_currentUS$_2016,HouseholdsFinalConsumptionExpenditure_constantUS$_2014,HouseholdsFinalConsumptionExpenditure_constantUS$_2015,HouseholdsFinalConsumptionExpenditure_constantUS$_2016
Country Code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AGO,Angola,2.692000e+07,2.786000e+07,2.881000e+07,32.160,23.570,18.570,1.922000e+09,9.282000e+09,4.104000e+09,4.703971,2.999747,-0.813494,7.279562,10.279327,34.736159,48.016371,33.448383,30.017037,1.267302e+11,1.026212e+11,9.533720e+10,1.008618e+11,1.038874e+11,1.030423e+11,42.245599,36.878451,29.417168,32.000000,42.000000,40.520607,48.279552,NaN,NaN,545.040491,NaN,NaN,34760.0,NaN,NaN,NaN,NaN,NaN,5.733624e+10,3.189484e+10,NaN,5.867237e+10,3.304778e+10,NaN,1.559736e+09,2.172579e+08,NaN,2.875350e+10,1.675779e+10,NaN,NaN,NaN,NaN,1.000154e+11,9.634418e+10,8.671644e+10,7.768358e+10,1.003854e+11,1.093904e+11,1.178803e+11,9.671368e+10,9.007774e+10,9.609642e+10,1.023279e+11,1.008467e+11,5.353793e+10,3.784512e+10,28045505039.2101,5.319462e+10,5.798212e+10,6.320050e+10,7.390546e+10,7.821012e+10,74434627139.9992,5.498982e+10,87022825584.3224,103223277005.568
ARE,United Arab Emirates,9.071000e+06,9.154000e+06,9.270000e+06,3.895,4.289,4.466,1.082000e+10,8.795000e+09,8.986000e+09,4.398697,5.064702,2.988442,2.345828,4.068834,1.617877,99.115041,100.870480,101.003014,4.031371e+11,3.581351e+11,3.570451e+11,3.550892e+11,3.730734e+11,3.842225e+11,68.935192,74.350827,75.744634,100.00

### Dropping useless columns 

In [156]:
# drop country name column
df.drop('Country Name', axis=1, inplace=True)

# dropping columns with too many missing values
def drop_poor_columns(dframe, p):
    ''' Drop the columns of dframe with more than p (%) missing values'''

    # create df with a the count of missing values for each column
    missing_df = pd.DataFrame(dframe.isnull().sum())
    # extract the names of columns with more than p (%) missing values
    poor_columns = missing_df.loc[missing_df[0] > p*len(dframe)].index
    # drop sparse columns
    dframe.drop(poor_columns, axis=1, inplace=True)
    return dframe, poor_columns

print('df.shape before', df.shape)
df, dropped = drop_poor_columns(df, 0.5)
print(dropped)

df.shape before (37, 81)
Index(['FossilFuelEnergyConsumption_%total_2015',
       'FossilFuelEnergyConsumption_%total_2016',
       'EnergyUse_KgEquivalentPerCapita_2015',
       'EnergyUse_KgEquivalentPerCapita_2016', 'CO2Emissions_kt_2015',
       'CO2Emissions_kt_2016',
       'PovertyHeadcountRatio$1.90aday_2011PPP_%population_2014',
       'PovertyHeadcountRatio$1.90aday_2011PPP_%population_2015',
       'PovertyHeadcountRatio$1.90aday_2011PPP_%population_2016',
       'AnnualHouseholdExpenditure_Gas_2010',
       'AnnualHouseholdExpenditure_Electricity_2010'],
      dtype='object')


In [157]:
# convert all columns to numeric type
df = df.apply(pd.to_numeric, errors='coerce')

### Dropping bad rows 

In [158]:
# dropping columns with too many missing values
def drop_poor_rows(dframe, p):
    ''' Drop the columns of dframe with more than p (%) missing values'''

    # create df with a the count of missing values for each column
    missing_df = pd.DataFrame(dframe.isnull().sum(axis=1))
    # extract the names of columns with more than p (%) missing values
    poor_rows = missing_df.loc[missing_df[0] > p*len(dframe.columns)].index
    # drop sparse columns
    dframe.drop(poor_rows, axis=0, inplace=True)
    return dframe, poor_rows

print('df.shape before', df.shape)
df, dropped = drop_poor_rows(df, 0.5)
print(dropped)

df.shape before (37, 70)
Index(['LBY', 'TKM', 'VEN'], dtype='object', name='Country Code')


In [159]:
# list of remaining countries by biggest number of missing data
df.isnull().sum(axis=1).sort_values(ascending=False)

Country Code
IRQ    25
TUN    16
GAB    15
TLS    14
COG    13
KEN    12
GHA    11
OMN    11
MAR     7
NGA     7
VNM     5
LBN     5
AGO     5
ECU     4
CHN     4
DZA     4
CIV     4
MNE     3
MOZ     3
PAK     3
IRL     2
AUS     1
CYP     1
ARE     1
ITA     1
EGY     1
GBR     1
NOR     1
PRT     1
USA     1
IDN     0
KAZ     0
MEX     0
ZAF     0
dtype: int64

### Imputing missing values

In [160]:
# df is my data frame with the missings. I keep only floats
df_numeric = df.select_dtypes(include=[np.float64]).as_matrix()

# I now run fancyimpute KNN, which returns a np.array:
# use 3 nearest rows which have a feature to fill in each row's missing features 
df_filled = pd.DataFrame(KNN(3).complete(df_numeric))

df_filled.columns = df.columns
df_filled.index = df.index

Imputing row 1/34 with 5 missing, elapsed time: 0.003


/home/lupol/tensorflow/local/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


### Exporting cleaned dataset

In [121]:
# export the cleaned dataframe to a csv file
df.to_csv('data/selected_countries_cleaned.csv')

# Principal Components Analysis

In [88]:
df = pd.read_csv('data/selected_countries_cleaned.csv').drop('Unnamed: 0', axis=1)
# change index (row labels)
df = df.set_index('Country Code', verify_integrity=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, AGO to ZAF
Data columns (total 70 columns):
Population_2014                                           37 non-null float64
Population_2015                                           37 non-null float64
Population_2016                                           37 non-null float64
Expense_%GDP_2014                                         21 non-null float64
Expense_%GDP_2015                                         21 non-null float64
Expense_%GDP_2016                                         19 non-null float64
ForeignDirectInvestment_NetInflows_BoP_currentUS$_2014    37 non-null float64
ForeignDirectInvestment_NetInflows_BoP_currentUS$_2015    37 non-null float64
ForeignDirectInvestment_NetInflows_BoP_currentUS$_2016    37 non-null float64
GDPgrowth_annual%_2014                                    37 non-null float64
GDPgrowth_annual%_2015                                    36 non-null float64
GDPgrowth_annual%_2016                   

In [89]:
df = scale(df)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [71]:
# scale dataframe before performing PCA
df = scale
X = pd.DataFrame(scale(df))
X.shape

ValueError: could not convert string to float: 'ZAF'